In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import time

from cuml.neighbors import KNeighborsRegressor as cuKNN
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('/content/drive/MyDrive/Machine Learning Project - 605/KMeans/yellow_tripdata.csv', header=None, low_memory=False)

In [ ]:
df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True)

In [ ]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2016-03-01 00:00:00,2016-03-01 00:07:55,1,2.50,-73.97674560546875,40.765151977539062,1,N,-74.004264831542969,40.746128082275391,1,9,0.5,0.5,2.05,0,0.3,12.35
1,1,2016-03-01 00:00:00,2016-03-01 00:11:06,1,2.90,-73.983482360839844,40.767925262451172,1,N,-74.005943298339844,40.733165740966797,1,11,0.5,0.5,3.05,0,0.3,15.35
2,2,2016-03-01 00:00:00,2016-03-01 00:31:06,2,19.98,-73.782020568847656,40.644809722900391,1,N,-73.974540710449219,40.675769805908203,1,54.5,0.5,0.5,8,0,0.3,63.8
3,2,2016-03-01 00:00:00,2016-03-01 00:00:00,3,10.78,-73.863418579101562,40.769813537597656,1,N,-73.969650268554688,40.757766723632812,1,31.5,0,0.5,3.78,5.54,0.3,41.62
4,2,2016-03-01 00:00:00,2016-03-01 00:00:00,5,30.43,-73.97174072265625,40.792182922363281,3,N,-74.177169799804688,40.695053100585937,1,98,0,0,0,15.5,0.3,113.8


In [ ]:
features = [
    'passenger_count', 'trip_distance', 'pickup_longitude', 'pickup_latitude',
    'dropoff_longitude', 'dropoff_latitude', 'fare_amount', 'extra',
    'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge'
]
target = 'total_amount'

In [ ]:
df = df.apply(pd.to_numeric, errors='coerce')

In [ ]:
X = df[features]
y = df[target]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [ ]:
import cuml
import cupy as cp

X_train_gpu = cp.asarray(X_train)
y_train_gpu = cp.asarray(y_train.values.astype(np.float32))

start_gpu = time.time()

knn_gpu = cuKNN(n_neighbors=5)
knn_gpu.fit(X_train_gpu, y_train_gpu)

end_gpu = time.time()
print(f"GPU Training Time: {end_gpu - start_gpu:.4f} seconds")


GPU Training Time: 2.1612 seconds


In [ ]:
X_test_gpu = cp.asarray(X_test)

start_gpu_pred = time.time()

y_pred_gpu = knn_gpu.predict(X_test_gpu)

end_gpu_pred = time.time()
print(f"GPU Prediction Time: {end_gpu_pred - start_gpu_pred:.4f} seconds")


GPU Prediction Time: 712.0004 seconds


In [ ]:
import cupy as cp
def print_memory_usage_gb():
  device = cp.cuda.Device(0)
  free_mem, total_mem = device.mem_info
  free_mem /= (1024 ** 3)
  total_mem /= (1024 ** 3)
  used_mem = total_mem - free_mem
  print(f"GPU VRAM Usage: {used_mem:.4f} GB out of {total_mem:.4f} GB")

print_memory_usage_gb()

GPU VRAM Usage: 1.6826 GB out of 14.7413 GB
